In [1]:
#Install Required Libraries
!pip install typer PyPDF2 spacy pandas numpy matplotlib seaborn scikit-learn sentence-transformers pytesseract pdf2image

# Install the SpaCy model
!python -m spacy download en_core_web_sm

!pip install wordcloud nltk
import nltk # Download WordNet data
nltk.download('wordnet')
nltk.download('omw-1.4')

!pip install -U sentence-transformers
!pip install shap
!pip install ipywidgets --upgrade
!pip install jupyter --upgrade
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

  Using cached pypdf2-3.0.1-py3-none-any.whl (232 kB)
  Using cached pdf2image-1.17.0-py3-none-any.whl (11 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.
pinecone-text 0.4.2 requires torch<2.0.0,>=1.13.1, but you have torch 2.0.1 which is incompatible.
pinecone-client 2.2.2 requires pyyaml>=5.4, but you have pyyaml 5.3.1 which is incompatible.
octoai-sdk 0.1.2 requires numpy<2.0.0,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
octoai-sdk 0.1.2 requires pyyaml<7.0,>=6.0,

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


  Using cached sentence_transformers-3.0.1-py3-none-any.whl (227 kB)
  Using cached transformers-4.43.3-py3-none-any.whl (9.4 MB)
  Using cached safetensors-0.4.3-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached huggingface_hub-0.24.3-py3-none-any.whl (417 kB)
  Using cached tokenizers-0.19.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.3.3
    Uninstalling safetensors-0.3.3:
      Successfully uninstalled safetensors-0.3.3
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.16.4
    Uninstalling huggingface-hub-0.16.4:
      Successfully uninstalled huggingface-hub-0.16.4
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: tran

In [1]:
import numpy as np
import pandas as pd
import os
import PyPDF2
import pytesseract
import re
import spacy

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from wordcloud import WordCloud
from nltk.corpus import wordnet as wn
from collections import defaultdict

from scipy.stats import chi2_contingency
from sklearn.metrics.pairwise import cosine_similarity
import shap
import torch

from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer, BertModel

2024-07-30 08:28:25.006747: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-30 08:28:25.008647: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 08:28:25.051605: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-30 08:28:25.052524: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-30 08:28:25.974494: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

### 1. Data Collection from open source website FOS
#### 1.1 Quarter batching run for 4 process steps : 
 step 1. web scrapping
 
 step 2. automated download for FOS' decision
 
 step 3. text extration from downloaded pdf files
 
 step 4. delete pdf files for the current quarter
 
Combine the results from step 3 in one dataframe.

In [37]:
%run scrape.py get-metadata-for-years --start-year 2015 --end-year 2018


Scraping 10 entries from page 0 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 10 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 20 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 30 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 40 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 50 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 60 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 70 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 80 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 90 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 100 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 110 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 120 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entries from page 130 for quarter 2015-01-01 to 2015-03-31
Scraping 10 entri

In [ ]:
%run scrape.py download-and-extract-text-for-years --start-year 2016 --end-year 2016

Skipping docfile/03_16_Insurance (excluding PPI)_DRN2928345.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN8505534.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN6645212.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN1095413.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN8643952.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN4697416.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN4445108.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN2013378.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN8722531.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN5137772.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN0397606.pdf as it already exists
Skipping docfile/03_16_Insurance (excluding PPI)_DRN0527493.pdf a

#### 1.2 Add decision and company names into data frame

In [ ]:
# Function to get metadata info
def get_metadata_info(filename):
    decision_id = re.search(r'DRN-(\d+)', filename).group(1)
    row = metadata[metadata['decision_id'].str.contains(decision_id)]
    if not row.empty:
        decision = row.iloc[0]['decision']
        company = row.iloc[0]['company']
        return decision, company
    else:
        return None, None
    
# Function to standardize company name with standard abbreviations
def standardize_company_name(company):
    # List of words to remove
    words_to_remove = ["Ltd", "ltd", "limited", "Limited", "Insurance", "Services", "Company", "Bank", "Plc", "UK", "U.K.", "Group", "Corporation", "Assurance", 
                       "Association", "International", "General", "Financial", "National", "Union","Consultant", "Consultants", "Society",
                       "Mutual", "Europe", "Europe SA/NV", "Europe SA", "Europe NV", "Europe SE", "SE", "NV"]

    # Standard abbreviations mapping
    abbreviations = {
        "Advantage Insurance": "Advantage",
        "AXA Insurance UK Plc": "AXA", "Allianz Insurance Plc": "Allianz", "Barclays Insurance": "Barclays",
        "Co-op Insurance": "Co-op", "Covea Insurance": "Covea", "Chubb Insurance": "Chubb", "Churchill Insurance": "Churchill",
        "Direct Line Insurance Group Plc": "Direct Line", 
        "Helvetia Global Solutions": "Helvetia",
        "HSBC Insurance": "HSBC", "John Lewis Insurance": "John Lewis",
        "Legal & General Insurance": "Legal & General", "Lloyds Bank Insurance": "Lloyds Bank", "LV Insurance": "(LV)",
        "Markerstudy Insurance": "Markerstudy", "Nationwide Insurance": "Nationwide", "Royal & Sun Alliance Insurance": "(RSA)",
        "Saga Insurance": "Saga", "Santander Insurance": "Santander", "Swinton Insurance": "Swinton", "Sainsbury's Insurance": "Sainsbury",
        "Tesco Bank Insurance": "Tesco Bank", "Virgin Money Insurance": "Virgin Money",
        "U K Insurance": "U K", "Zurich Insurance Plc": "Zurich"
    }

    # Remove the words_to_remove from the company name
    pattern = re.compile(r'\b(' + '|'.join(words_to_remove) + r')\b', re.IGNORECASE)
    standardized_name = pattern.sub('', company).strip()
    
    # Apply abbreviations
    if standardized_name in abbreviations:
        standardized_name = abbreviations[standardized_name]
    
    return standardized_name

# Adding decision and company columns to the DataFrame
def add_decision_and_company_info(df, metadata):
    decisions = []
    companies = []
    for filename in df['filename']:
        decision, company = get_metadata_info(filename)
        decisions.append(decision)
        companies.append(standardize_company_name(company))
    
    df['decision'] = decisions
    df['company'] = companies

    # Remove blank brackets () from company names
    df['company'] = df['company'].str.replace(r'\(\s*\)', '', regex=True).str.strip()

    return df

In [ ]:
current_directory = os.getcwd()

# Load metadata
metadata = pd.read_csv(os.path.join(current_directory, 'metadata.csv'))

# Apply the function to add decision and company info to the DataFrame
cdf = add_decision_and_company_info(combined_df, metadata)

In [36]:
metadata_x = pd.read_csv(os.path.join(current_directory, 'metadata.csv'))
metadata_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5605 entries, 0 to 5604
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   decision_id  5605 non-null   object 
 1   location     5605 non-null   object 
 2   title        5605 non-null   object 
 3   date         5605 non-null   object 
 4   company      5605 non-null   object 
 5   decision     5605 non-null   object 
 6   extras       0 non-null      float64
 7   tag          5605 non-null   object 
dtypes: float64(1), object(7)
memory usage: 350.4+ KB


In [3]:
current_directory = os.getcwd()

# Load metadata
df_upto2015 = pd.read_csv(os.path.join(current_directory, 'extracted_texts_all_years.csv'))
df_upto2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8793 entries, 0 to 8792
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   filename       8793 non-null   object
 1   original_text  8793 non-null   object
 2   period         8793 non-null   object
 3   period_date    8793 non-null   object
dtypes: object(4)
memory usage: 274.9+ KB


In [4]:
df_upto2015['period'].unique()

array(['06_13', '05_13', '04_13', '09_13', '08_13', '07_13', '12_13',
       '11_13', '10_13', '03_14', '02_14', '01_14', '06_14', '05_14',
       '04_14', '09_14', '08_14', '07_14', '12_14', '11_14', '10_14'],
      dtype=object)

In [5]:
Q12015 = pd.read_csv(os.path.join(current_directory, 'extracted_texts_2015_Q1.csv'))
Q22015 = pd.read_csv(os.path.join(current_directory, 'extracted_texts_2015_Q2.csv'))
Q32015 = pd.read_csv(os.path.join(current_directory, 'extracted_texts_2015_Q3.csv'))
Q42015 = pd.read_csv(os.path.join(current_directory, 'extracted_texts_2015_Q4.csv'))

df2015 = pd.concat([df_upto2015, Q12015], ignore_index=True)
df2015 = pd.concat([df2015, Q22015], ignore_index=True)
df2015 = pd.concat([df2015, Q32015], ignore_index=True)
df2015 = pd.concat([df2015, Q42015], ignore_index=True)
df2015.info()
#df2015.to_csv('extracted_texts_all_years.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15745 entries, 0 to 15744
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   filename       15745 non-null  object
 1   original_text  15745 non-null  object
 2   period         8793 non-null   object
 3   period_date    8793 non-null   object
dtypes: object(4)
memory usage: 492.2+ KB


In [6]:
df2015.to_csv('extracted_texts_all_years.csv', index=False)

In [7]:
df_upto2015 = pd.read_csv(os.path.join(current_directory, 'extracted_texts_all_years.csv'))
df_upto2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15745 entries, 0 to 15744
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   filename       15745 non-null  object
 1   original_text  15745 non-null  object
 2   period         8793 non-null   object
 3   period_date    8793 non-null   object
dtypes: object(4)
memory usage: 492.2+ KB


### 2. Natural Language Processing for Data Extraction
Textual data needs to be further analysed to extract relevant information from each document, the data extracted can be
used as features for a downstream modelling task.